# Pivot Table Construction

In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
trans_detail = pd.read_excel("Find Journal Lines - NMSS 2020-08-11 13_55 PDT.xlsx", sheet_name='Sheet1')

Wall time: 1min 27s


In [3]:
FY20_Data = trans_detail.drop(trans_detail.index[0:35])

FY20_Data = FY20_Data.reset_index()

FY20_Data = FY20_Data.drop('index',1)

FY20_Data.rename(columns=FY20_Data.iloc[1])

#print(FY20_Data.iloc[0])

#print(FY20_Data.head())

column_names = ['Journal', 'Journal Number', 'Journal Sequence Number', 'Journal Memo', 'Operational Transaction', 'Originated by', 'Approved by', 'Company',
'Intercompany Initiating Company','Status', 'Accounting Date','Posted Date','Source', 'Ledger','Currency','Ledger Account','Ledger Debit Amount',
'Ledger Credit Amount','Line Memo','External Reference ID', 'Spend Category','Revenue Category', 'Cost Center','Fund',
'Gift','Program','Project','Region','Location','Fundraising Type', 'InKind','State','Bank Account','Intercompany Affiliate','Supplier','Customer','Employee','Match ID']

FY20_Data.columns = column_names

FY20_Data = FY20_Data.drop([0])

FY20_Data = FY20_Data.reset_index()

FY20_Data = FY20_Data.drop('index',1)

In [4]:
FY20_Data['Debit-Credit'] = FY20_Data.apply(lambda x: x['Ledger Debit Amount'] - x['Ledger Credit Amount'], axis=1)
FY20_Data['Accounting Date'] = pd.to_datetime(FY20_Data['Accounting Date'])
FY20_Data['Month - Yr'] = FY20_Data['Accounting Date'].apply(lambda x: x.strftime('%m-%y'))
GL_Lookup = pd.read_excel('Ledger_Account_Table.xlsx', sheet_name = 'Sheet1')
FY20_Data = pd.merge(FY20_Data, GL_Lookup, on='Ledger Account', how='left')

In [5]:
def return_first(x):
    first = x[0]
    return first

In [6]:
FY20_Data['First'] = FY20_Data['Ledger Account'].apply(return_first)

In [7]:
FY20_Data['Exp/DBD/Rev'] = FY20_Data['First'].map({'4':'Revenue', '5':'Expense', '6':'DBD', '1':'BS', '2':'BS', '3':'BS'})

In [8]:
del FY20_Data['First']

In [9]:
FY20_Data['Debit-Credit'].fillna(0, inplace=True)

FY20_Data['Supplier'].fillna("Not Available", inplace=True)

FY20_Data['Line Memo'].fillna("No Description Available", inplace=True)

FY20_Data['Operational Transaction'].fillna("Not Available", inplace=True)

FY20_Data['Project'].fillna("No Project", inplace=True)

FY20_Data['Spend Category'].fillna("No Spend Category", inplace=True)

FY20_Data['Location'].fillna("No Location", inplace=True)

In [10]:
Cost_Centers = list(FY20_Data['Cost Center'].unique())


print(Cost_Centers)

[nan, 'CC014 Finance Operations', 'CC023 Bike MS', 'CC005 Facilities', 'CC025 Other Special Events', 'CC012 Technology', 'CC024 Walk MS', 'CC006 Human Resources', 'CC029 Professional Education', 'CC022 Leadership Events', 'CC018 Direct Marketing\xa0', 'CC009 Marketing', 'CC004 Operations', 'CC027 Programs & Services', 'CC001 Research', 'CC028 MS Navigator', 'CC016 Individual Giving', 'CC021 Challenge Walk', 'CC020 Do It Yourself (DIY)', 'CC019 Corporate Relations', 'CC002 Advocacy', 'CC032 Services Resources', 'CC033 Community Leadership', 'CC015 Development', 'CC026 MuckFest MS', 'CC007 Volunteer Engagement', 'CC010 Digital Marketing', 'CC008 Legal and Risk', 'CC003 ELT', 'CC034 Community Engagement', 'CC035 Event Production', 'CC017 Tools and Resources', 'CC011 Brand Management', 'CC013 Strategic Finance', 'CC036 Fundraising Support']


In [11]:
def createCC(Cost_Center):
    return FY20_Data[(FY20_Data['Cost Center']==Cost_Center) & (FY20_Data['In/Out']=='In') & (FY20_Data['Exp/DBD/Rev']=='Expense')]
 
    

In [12]:
def create_Occupancy(Cost_Center):
        return FY20_Data[(FY20_Data['Cost Center']==Cost_Center) & (FY20_Data['In/Out']=='In') & (FY20_Data['Exp/DBD/Rev']=='Expense') & (FY20_Data['Spend Category']=='Office Cleaning & CAM Expense')|(FY20_Data['Spend Category']=='Office Moving Expense')|(FY20_Data['Spend Category']=='Office Rent Expense')|(FY20_Data['Spend Category']=='Property Tax Expense')|(FY20_Data['Spend Category']=='Security Expense')|(FY20_Data['Spend Category']=='Staff Relocation Expense')|(FY20_Data['Spend Category']=='Storage Rent Expense')|(FY20_Data['Spend Category']=='Utilities')]

In [13]:
Occupancy = create_Occupancy('CC005 Facilities')

In [14]:
FinanceOps = createCC('CC014 Finance Operations')
StrategicFin = createCC('CC013 Strategic Finance')

Finance = pd.concat([FinanceOps, StrategicFin], ignore_index=True, sort=False)


In [15]:
Ops = createCC('CC004 Operations')
ELT = createCC('CC003 ELT')

Operations = pd.concat([Ops, ELT], ignore_index=True, sort=False)

In [16]:
HR = createCC('CC006 Human Resources')
VE = createCC('CC007 Volunteer Engagement')
Legal = createCC('CC008 Legal and Risk')
Professional_Education = createCC('CC029 Professional Education')
Direct_Marketing = createCC('CC018 Direct Marketing\xa0')
Marketing = createCC('CC009 Marketing')
Research = createCC('CC001 Research')
MS_Navigator = createCC('CC028 MS Navigator')
Development = createCC('CC015 Development')
Digital_Marketing = createCC('CC010 Digital Marketing')
Community_Engagement = createCC('CC034 Community Engagement')
Event_Production = createCC('CC035 Event Production')
Tools_and_Resources = createCC('CC017 Tools and Resources')
Brand_Management = createCC('CC011 Brand Management')
Fundraising_Support = createCC('CC036 Fundraising Support')
Facilities = createCC('CC005 Facilities')

In [17]:
Technology = createCC('CC012 Technology')

In [18]:
def create_project_pivot(Group):
    Month_order = ['10-19', '11-19', '12-19', '01-20', '02-20', '03-20', '04-20', '05-20', '06-20', '07-20', '08-20', '09-20']
    table = pd.pivot_table(Group, index = ['Project','Spend Category','Supplier','Line Memo', 'Operational Transaction'], values = 'Debit-Credit', columns = 'Month - Yr', aggfunc=np.sum, fill_value = 0)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:-1].sum(axis=1)
    return table2

In [19]:
def create_Occupancy_pivot(Group):
    Month_order = ['10-19', '11-19', '12-19', '01-20', '02-20', '03-20', '04-20', '05-20', '06-20', '07-20', '08-20', '09-20']
    table = pd.pivot_table(Group, index = ['Location','Spend Category','Supplier','Line Memo', 'Operational Transaction'], values = 'Debit-Credit', columns = 'Month - Yr', aggfunc=np.sum, fill_value = 0)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:-1].sum(axis=1)
    return table2

In [20]:
Occupancypivot = create_Occupancy_pivot(Occupancy)

In [21]:
Technologypivot = create_project_pivot(Technology)

In [22]:
def create_basic_pivot(Group):
    Month_order = ['10-19', '11-19', '12-19', '01-20', '02-20', '03-20', '04-20', '05-20', '06-20', '07-20', '08-20', '09-20']
    table = pd.pivot_table(Group, index = ['Spend Category','Supplier','Line Memo', 'Operational Transaction'], values = 'Debit-Credit', columns = 'Month - Yr', aggfunc=np.sum, fill_value = 0)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:-1].sum(axis=1)
    return table2

In [23]:
import os

path = r'C:\Users\jeremy.wendt\Anaconda3\envs\pandas_trans\Details'

In [24]:
Financepivot = create_basic_pivot(Finance)
VEpivot = create_basic_pivot(VE)
HRpivot = create_basic_pivot(HR)
Legalpivot = create_basic_pivot(Legal)
Prof_Edpivot = create_basic_pivot(Professional_Education)
DirectMarketingpivot = create_basic_pivot(Direct_Marketing)
Marketingpivot = create_basic_pivot(Marketing)
Researchpivot = create_basic_pivot(Research)
MS_Navigatorpivot = create_basic_pivot(MS_Navigator)
Developmentpivot = create_basic_pivot(Development)
Digital_Marketingpivot = create_basic_pivot(Digital_Marketing)
Community_Engagementpivot = create_basic_pivot(Community_Engagement)
Event_Productionpivot = create_basic_pivot(Event_Production)
ToolsandResourcespivot = create_basic_pivot(Tools_and_Resources)
Brand_Managementpivot = create_basic_pivot(Brand_Management)
FundraisingSupportpivot = create_basic_pivot(Fundraising_Support)
Facilitiespivot = create_basic_pivot(Facilities)


In [25]:
def add_subtotal(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Spend Category')]).append((PivotedGroup.sum()).rename(('Grand', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [26]:
def add_subtotal_Project(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Project')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [27]:
def add_subtotal_Occupancy(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Location')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [28]:
Occupancypivot = add_subtotal_Occupancy(Occupancypivot)

In [29]:
Technologypivot = add_subtotal_Project(Technologypivot)

In [30]:
Financepiv = add_subtotal(Financepivot)
HRpivot = add_subtotal(HRpivot)
VEpivot = add_subtotal(VEpivot)
Legalpivot = add_subtotal(Legalpivot)
Prof_Edpivot = add_subtotal(Prof_Edpivot)
DirectMarketingpivot = add_subtotal(DirectMarketingpivot)
Marketingpivot = add_subtotal(Marketingpivot)
Researchpivot = add_subtotal(Researchpivot)
MS_Navigatorpivot = add_subtotal(MS_Navigatorpivot)
Developmentpivot = add_subtotal(Developmentpivot)
Digital_Marketingpivot = add_subtotal(Digital_Marketingpivot)
Community_Engagementpivot = add_subtotal(Community_Engagementpivot)
Event_Productionpivot = add_subtotal(Event_Productionpivot)
ToolsandResourcespivot = add_subtotal(ToolsandResourcespivot)
Brand_Managementpivot = add_subtotal(Brand_Managementpivot)
FundraisingSupportpivot = add_subtotal(FundraisingSupportpivot)
Facilitiespivot = add_subtotal(Facilitiespivot)

In [31]:
def to_excel(pivot, str):
    pivot.to_excel(path+str, freeze_panes = (1,4), engine = 'openpyxl', sheet_name = 'Trans Detail', float_format = "%.0f")

In [32]:
def to_excel_Project(pivot, str):
    pivot.to_excel(path+str, freeze_panes = (1,5), engine = 'openpyxl', sheet_name = 'Trans Detail', float_format = "%.0f")

In [33]:
to_excel_Project(Technologypivot,'\Technology.xlsx')
to_excel_Project(Occupancypivot,'\Occupancy.xlsx')

In [34]:

file_names = ['\Facilities.xlsx', '\Brand_Management.xlsx', '\VE.xlsx', '\HR.xlsx', '\Finance.xlsx', '\Legal.xlsx', '\Prof_Ed.xlsx', '\DirectMarketing.xlsx', '\Marketing.xlsx', '\Research.xlsx', '\MS_Navigator.xlsx', '\Development.xlsx', '\Digital_Marketing.xlsx', '\Community_Engagement.xlsx', '\Event_Production.xlsx', '\FundraisingSupport.xlsx'] 

to_excel(VEpivot,'\VE.xlsx')
to_excel(Financepiv,'\Finance.xlsx')
to_excel(Legalpivot,'\Legal.xlsx')
to_excel(HRpivot,'\HR.xlsx')
to_excel(Prof_Edpivot,'\Prof_Ed.xlsx')
to_excel(DirectMarketingpivot,'\DirectMarketing.xlsx')
to_excel(Marketingpivot,'\Marketing.xlsx')
to_excel(Researchpivot,'\Research.xlsx')
to_excel(MS_Navigatorpivot,'\MS_Navigator.xlsx')
to_excel(Developmentpivot,'\Development.xlsx')
to_excel(Digital_Marketingpivot,'\Digital_Marketing.xlsx')
to_excel(Community_Engagementpivot,'\Community_Engagement.xlsx')
to_excel(Event_Productionpivot,'\Event_Production.xlsx')
to_excel(ToolsandResourcespivot,'\ToolsandResources.xlsx')
to_excel(Brand_Managementpivot,'\Brand_Management.xlsx')
to_excel(FundraisingSupportpivot,'\FundraisingSupport.xlsx')
to_excel(Facilitiespivot,'\Facilities.xlsx')







In [36]:
from openpyxl.styles import Color, PatternFill, Font, Border
greyFill = PatternFill(start_color='00C0C0C0',
                   end_color='00C0C0C0',
                   fill_type='solid')

In [37]:
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side


def edit_workbook(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Spend Category"
    sheet['B1']= "Supplier"
    sheet['C1']= "Line Memo"
    sheet['D1']= "Operational Transaction"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(5, 20):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0.00'
    for col in range(1, 5):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=4).value == "Subtotal":
            for j in range(5,18):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    book.save(file)

In [38]:
def edit_workbook_projects(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Project"
    sheet['B1']= "Spend Category"
    sheet['C1']= "Supplier"
    sheet['D1']= "Line Memo"
    sheet['E1']= "Operational Transaction"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(6, 21):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0.00'
    for col in range(1, 6):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill            
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=5).value == "Subtotal":
            for j in range(6,19):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    book.save(file)

In [39]:
def edit_workbook_Occupancy(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Location"
    sheet['B1']= "Spend Category"
    sheet['C1']= "Supplier"
    sheet['D1']= "Line Memo"
    sheet['E1']= "Operational Transaction"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(6, 21):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0.00'
    for col in range(1, 6):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill              
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=5).value == "Subtotal":
            for j in range(6,19):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    book.save(file)

In [40]:
edit_workbook_projects(path+'\Technology.xlsx')
edit_workbook_Occupancy(path+'\Occupancy.xlsx')


In [41]:
%%time
for i in file_names:
    edit_workbook(path+i)

Wall time: 46.8 s
